In [1]:
G_Debug = True   # if debug mode.  Debug = True. if Release mode, Debug = False.
G_Chrome_Hide = True
G_Debug_Pages = 2
G_Debug_Items = 2


G_CVS_FILE_NAME =  'tianjin.csv'

G_XLSX_FILE_NAME = 'tianjin2019.xlsx'

G_BID_ATTRIBUTE = ("id","status","start","end","title","consultPrice",
                                "marketPrice","currentPrice","bidCount","delayCount","applyCount","itemUrl")

G_CVS_HEADER =  ("name","start","end","bidNumber","title","originPrice",
                         "currentPrice","discount","unit","bidCount","delayCount",
                         "applyCount","area","areaA","areaB","itemUrl")


In [2]:
#will bid
# sourceUrl = ("https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.60.53db26cdCGMLqt&category=50025969"  
#                    "&auction_source=0&province=%CC%EC%BD%F2&sorder=1&st_param=-1&auction_start_seg=-1")
# #done bid
sourceUrl = ("https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2"
               "&st_param=-1&auction_start_seg=&auction_start_from=2019-01-01&auction_start_to=2019-07-10&&spm=a213w.3064813.9001.2")




In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import json
import time
import csv
import pandas as pd
import re

In [4]:
# time
def getCurrentTimeStamp():
    t = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t)))
    return t


def printDuration(start, end):
    seconds = end - start
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    print ("%02d:%02d:%02d" % (h, m, s))
    
    

# import time
# dt = "2019-06-03 10:00:00"
# #转换成时间数组
# timeArray = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
# #转换成时间戳
# timestamp = time.mktime(timeArray)
# print (timestamp)

In [ ]:
def log(page_id,item_id,status):
    print("page id: {0}, item id: {1}, status:{2}".format(page_id,item_id,status))

In [ ]:
class DataManagement:
    def __init__(self):
        self.data = {}
        self.data["name"] = None
        
        global G_CVS_FILE_NAME
        global G_XLSX_FILE_NAME
        global G_CVS_HEADER
        
        self.cvs_file_name = G_CVS_FILE_NAME
        self.xlsx_file_name = G_XLSX_FILE_NAME
        self.cvs_head = G_CVS_HEADER
        
        self.__open_file()

    def __process_attri_name(self,attribute, data):
        index = data.find("通过")
        data = data[4:index]
        self.data[attribute] =  data     
            
    def __process_attri_area(self,attribute, data):
        left = data.find("建筑面积")
        if(left != -1):
            size = data[(left+4) : (left+22)]
            areas = re.findall(r"\d+\.\d*",size)
            self.data[attribute] = areas[0] if areas else 0
        else:
            self.data[attribute] = 0
        
    def __process_attri_date(self,attribute, data):
        time1 = int(int(data)/1000 )
        self.data[attribute] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time1)))
        
    def __process_attri_times(self,attribute, data):
        keyword_postion = data.find("卖")
        if(keyword_postion != -1):
            data = data[1:keyword_postion+1]
        else:
            data = "no content"
            
        self.data[attribute] = data



    def __process_attri_common(self,attribute, data):
        self.data[attribute] =  data        
     
    def __process_attri_url(self,attribute, data):
        url = "https:"+data
        self.data[attribute] =  url        

    
    def __clac_data(self):
        # 
        if (int(self.data["consultPrice"]) > 0):
            originPrice = self.data["consultPrice"]
        else:
            originPrice = self.data["marketPrice"]
        #
        if (int(originPrice)  == 0):
            self.data["discount"] = "error"
        else:
            self.data["discount"]  = float("%.2f" % float(int(self.data["currentPrice"])/originPrice))
 
       # change unit to from RMB yuan to  RMB Wan
        self.data["originPrice"]  = float("%.2f" % float(int(originPrice)/10000))        
        self.data["currentPrice"] = float("%.2f" % float(int(self.data["currentPrice"])/10000))

      # set area
        if(float(self.data["areaA"]) > 0):
            self.data["area"]  = self.data["areaA"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        elif(float(self.data["areaB"]) > 0):
            self.data["area"] = self.data["areaB"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        else:
            self.data["area"] = 0
            self.data["unit"] = 0
      ###    
        
    
    def set_data(self,attribute, data):
        if(attribute == "name"):
            self.__process_attri_name(attribute, data)
        elif(attribute == "areaA" or attribute == "areaB"):
            self.__process_attri_area(attribute, data)
        elif(attribute == "start" or attribute == "end"):
            self.__process_attri_date(attribute, data)
        elif(attribute == "bidNumber"):
            self.__process_attri_times(attribute, data)
        elif(attribute == "itemUrl"):
            self.__process_attri_url(attribute, data)
        else:
            self.__process_attri_common(attribute,data)    
    
    
    def get_data(self,attribute):
        return self.data[attribute]
    
    def __open_file(self):
        self.csv_file = open(self.cvs_file_name,"w+")
        self.writer = csv.writer(self.csv_file)        
        self.writer.writerow(self.cvs_head)

    def write_file(self):
        
        self.__clac_data()
        
        row = tuple(self.data[i] for i in self.cvs_head)
        
        self.writer.writerow(row)
    
    def close_file(self):
        self.csv_file.close()
        csv = pd.read_csv(self.cvs_file_name, encoding='utf-8')
        csv.to_excel(self.xlsx_file_name, sheet_name='data')
            


In [ ]:
class SfSpider:
    def __init__(self,head_less,debug,url):
        
        global G_Debug_Pages
        global G_Debug_Items
        global G_BID_ATTRIBUTE
        self.debug_pages = G_Debug_Pages
        self.debug_items = G_Debug_Items
        
        
        self.data = DataManagement()
        
        if(head_less):
            #set headless
            chrome_options=Options()
            chrome_options.add_argument('--headless')
            self.driver = webdriver.Chrome(options=chrome_options)
        else:
            self.driver = webdriver.Chrome()
            
        self.debug = debug
        self.url = url
        # Setup wait for later
        self.wait = WebDriverWait(self.driver, 10)

        
    def __wait_and_click(self,mode,value):
    #  Sometimes click fails unreasonably. So tries to click at all cost.
        try:
            if(mode == "LINK_TEXT"):
                elem = self.wait.until(EC.element_to_be_clickable((By.LINK_TEXT, value)))
            elif(mode == "CSS_SELECTOR"):
                elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,value)))
            elif(mode == "XPATH"):
                elem = self.wait.until(EC.element_to_be_clickable((By.XPATH,value)))
            else:
                print('wait_and_click() error...')

                
            elem.click()
            time.sleep(1)
        except Exception as e:
            print('Click time out - {}'.format(value))
            print('Refreshing browser...')
            self.driver.refresh()
            time.sleep(2)
            return self.__wait_and_click(mode,value)

        return elem
        
        
    def do_crawling(self):
        
        self.driver.get(self.url)
        
        assert len(self.driver.window_handles) == 1

        elem = self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))
        
        if(self.debug):
            self.pages = self.debug_pages
        else:
            self.pages = int(elem.find_element_by_class_name("page-total").text)
            
        self.__do_page_crawling()

        

    def __do_page_crawling(self):
        for page in range(1,self.pages+1):
            if(page > 1):
                self.__wait_and_click("LINK_TEXT",str(page))
            
            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
            jsonData =  json.loads(self.driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
            page_data = jsonData["data"]

            if(self.debug):
                self.items = self.debug_items
            else:
                self.items = len(page_data)    

            self.__do_item_crawling(page,page_data)
            
        self.driver.quit()
        self.data.close_file()
        

    def __do_item_crawling(self,page_id,page_data):
        for item in range(self.items):
            log(page_id,item,page_data[item]["status"])
            for key,value in page_data[item].items():
                if key in G_BID_ATTRIBUTE:
                    self.data.set_data(key,value)

            self.__wait_and_click("CSS_SELECTOR","#pai-item-"+str(self.data.get_data("id")))

            # Wait for the new window or tab
            self.wait.until(EC.number_of_windows_to_be(2))
            handles = self.driver.window_handles
            self.driver.switch_to.window(handles[1])

           # bid times
            self.wait.until(EC.presence_of_element_located((By.XPATH, "/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]")))
            self.data.set_data("bidNumber",self.driver.find_element_by_xpath("/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]").get_attribute('textContent'))
            #TODO: degfine new field

           # 变卖公告，竞买公告
            self.__wait_and_click("CSS_SELECTOR","#J_DetailTabMenu > li.current.first > a")

            self.data.set_data("areaA",self.driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))


            self.__wait_and_click("LINK_TEXT","标的物介绍")

            self.data.set_data("areaB",self.driver.find_element_by_id("J_desc").get_attribute('textContent'))


            if(self.data.get_data("status") == "done"):

                self.__wait_and_click("LINK_TEXT","竞价成功确认书")

                self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))

                name = self.driver.find_element_by_class_name("c-content").text
            elif(self.data.get_data("status") == "failure"):
                name = "用户姓名流拍通过"
            else:
                name = "用户姓名即将开始通过"
            self.data.set_data("name",name)

            self.data.write_file()
            self.driver.close()
            self.driver.switch_to.window(handles[0])

In [ ]:
if __name__ == '__main__':
    startTime =  getCurrentTimeStamp()
    
    sf = SfSpider(G_Chrome_Hide,G_Debug,sourceUrl)
    sf.do_crawling()
    
    endTime = getCurrentTimeStamp()
    
    printDuration(startTime, endTime)

2019-06-19 07:05:38
